In [5]:
import pandas as pd 
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle

In [2]:
df=pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df=df.drop(["RowNumber","CustomerId","Surname"],axis=1)

In [7]:
label_encoder_gender=LabelEncoder()
df['Gender']=label_encoder_gender.fit_transform(df["Gender"])
onehot_encoder_geo=OneHotEncoder(handle_unknown="ignore")
geo_encoded=onehot_encoder_geo.fit_transform(df[["Geography"]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(["Geography"]))


In [8]:
df=pd.concat([df.drop("Geography",axis=1),geo_encoded_df],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [9]:
with open("label_encoder_gender.pkl",'wb') as file:
    pickle.dump(label_encoder_gender,file)

with open("onehot_encoder_geo.pkl","wb") as file:
    pickle.dump(onehot_encoder_geo,file)

In [11]:
X=df.drop("EstimatedSalary",axis=1)
y=df["EstimatedSalary"]

## train test split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

#scale the data

Scaler=StandardScaler()
X_train=Scaler.fit_transform(X_train)
X_test=Scaler.transform(X_test)
with open("scaler.pkl","wb") as file:
    pickle.dump(Scaler,file)

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

2025-12-26 20:56:50.618169: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-26 20:56:50.699353: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-26 20:56:50.699422: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-26 20:56:50.701214: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-26 20:56:50.714383: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-26 20:56:50.714965: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [13]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
])
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
import datetime
log_dir='regressionlogs/fit/'+datetime.datetime.now().strftime("%Y%m%d.%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)
early_stopping_callback=EarlyStopping(monitor="val_loss",patience=20,restore_best_weights=True)

In [20]:
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=(tensorflow_callback,early_stopping_callback)
)

Epoch 1/100
  1/250 [..............................] - ETA: 5s - loss: 113005.9531 - mae: 113005.9531

250/250 [==============================] - 1s 6ms/step - loss: 100430.2578 - mae: 100430.2578 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 100430.3359 - mae: 100430.3359 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 3/100
250/250 [==============================] - 1s 5ms/step - loss: 100430.3047 - mae: 100430.3047 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 100430.2969 - mae: 100430.2969 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 100430.2969 - mae: 100430.2969 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 6/100
250/250 [==============================] - 2s 7ms/step - loss: 100430.2578 - mae: 100430.2578 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 100430.2734 - mae: 100430.2734 

In [22]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [23]:
%reload_ext tensorboard
%tensorboard --logdir {log_dir}

Reusing TensorBoard on port 6010 (pid 22348), started 0:00:12 ago. (Use '!kill 22348' to kill it.)

In [25]:
test_loss,test_mae=model.evaluate(X_test,y_test)
print("Test MAE : ",{test_loss})

63/63 [==============================] - 0s 3ms/step - loss: 98725.0625 - mae: 98725.0625
Test MAE :  {98725.0625}


In [26]:
model.save('regressionmodel.h5')

/home/rguktrkvalley/ANN Classification/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
